In [26]:
from haystack import Document, component
from haystack.components.builders import PromptBuilder
from haystack.components.retrievers import InMemoryEmbeddingRetriever
from haystack.components.generators import OpenAIGenerator
from haystack.components.generators.openai_utils import _convert_message_to_openai_format
from haystack.dataclasses import ChatMessage, StreamingChunk
from haystack.document_stores.in_memory import InMemoryDocumentStore
# from haystack_integrations.components.embedders.cohere import CohereDocumentEmbedder, CohereTextEmbedder
from haystack.components.embedders import OpenAITextEmbedder, OpenAIDocumentEmbedder
from haystack_integrations.document_stores.chroma import ChromaDocumentStore

from datasets import load_dataset
from openai import OpenAI, Stream
from openai.types.chat import ChatCompletion, ChatCompletionChunk
from typing import List, Any, Dict, Optional, Callable, Union
from pydantic import BaseModel



document_store = ChromaDocumentStore(persist_path="got_chroma_db")

dataset = load_dataset("Tuana/game-of-thrones", split="train")
docs = [Document(content=doc["content"], meta=doc["meta"]) for doc in dataset]

doc_embedder = OpenAIDocumentEmbedder(model="text-embedding-3-small")
docs_with_embeddings = doc_embedder.run(docs)
document_store.write_documents(docs_with_embeddings["documents"])

Calculating embeddings: 100%|██████████| 74/74 [01:27<00:00,  1.19s/it]
Add of existing embedding ID: 097c832f6c7ced24fde708966b9e67b99cbccb47e28a8f68b8e62c49d9b2d1fc
Insert of existing embedding ID: 097c832f6c7ced24fde708966b9e67b99cbccb47e28a8f68b8e62c49d9b2d1fc
Add of existing embedding ID: c7ba6ece7500b20c8bfaa9968a0bd304ff754cdaa6b039004fe909c359102b43
Insert of existing embedding ID: c7ba6ece7500b20c8bfaa9968a0bd304ff754cdaa6b039004fe909c359102b43
Add of existing embedding ID: 412f475080d20cc24c51a7a7c4114810bee9825ef451fddbc9230eef6d2f090e
Insert of existing embedding ID: 412f475080d20cc24c51a7a7c4114810bee9825ef451fddbc9230eef6d2f090e
Add of existing embedding ID: e82cf2f624533f833bcecf77667cfca4a5f1bf32421aa6088bb24536ef0abdba
Insert of existing embedding ID: e82cf2f624533f833bcecf77667cfca4a5f1bf32421aa6088bb24536ef0abdba
Add of existing embedding ID: 815873730d2e63f355bf2d9dd83bae3db6ca14ed7f3cae7ec81ab43ae5c89b2c
Insert of existing embedding ID: 815873730d2e63f355bf2d9dd83b

2357

In [61]:
from haystack_integrations.components.retrievers.chroma import ChromaEmbeddingRetriever

# Initialize components
# retriever = InMemoryEmbeddingRetriever(document_store=document_store, top_k=3)  # Limit to top 3 results
retriever = ChromaEmbeddingRetriever(document_store=document_store, top_k=3)
text_embedder = OpenAITextEmbedder(model="text-embedding-3-small")
generator = OpenAIGenerator()

# Build prompt template
template = """Answer the question based on the provided context and chat history. If you cannot find the answer in the context, say so.

Previous conversation:
{% for message in chat_history %}
{% if message.role == 'user' %}User: {{ message.content }}{% endif %}
{% if message.role == 'assistant' %}Assistant: {{ message.content }}{% endif %}
{% endfor %}

Context:
{% for doc in documents %}
{{ doc.content }}
{% endfor %}

Question: {{question}}

Answer:"""

prompt_builder = PromptBuilder(template=template)

class ChatSession:
    def __init__(self):
        self.chat_history = []
        self.context_history = []
        
    def chat(self, question: str):
        # Embed question and retrieve relevant docs
        embedded_query = text_embedder.run(question)
        retrieved_docs = retriever.run(embedded_query["embedding"])
        
        # Store context
        self.context_history.extend(retrieved_docs["documents"])
        
        # Build prompt with retrieved context and chat history
        prompt_params = {
            # "documents": retrieved_docs["documents"], 
            "documents": self.context_history,
            "question": question,
            "chat_history": self.chat_history
        }
        prompt = prompt_builder.run(**prompt_params)
        print(prompt)
        
        # Generate answer
        response = generator.run(prompt["prompt"])
        answer = response["replies"][0]
        
        # Update chat history
        self.chat_history.append(ChatMessage.from_user(question))
        self.chat_history.append(ChatMessage.from_assistant(answer))
        
        return f"{answer}\n\nSources: {', '.join(list(set([i.meta["name"] for i in session.context_history])))}" #output the document names

# Gradio chat interface
import gradio as gr

session = ChatSession()

def chat_response(message, history):
    return session.chat(message)

demo = gr.ChatInterface(
    fn=chat_response,
    title="Game of Thrones Chat", 
    description="Ask questions about Game of Thrones",
)

demo.launch(show_api=False, share=False)


/Users/adamfletcher/Documents/GitHub/find-techspike-rag/.venv/lib/python3.12/site-packages/gradio/components/chatbot.py:229: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7883

To create a public link, set `share=True` in `launch()`.


{'prompt': "Answer the question based on the provided context and chat history. If you cannot find the answer in the context, say so.\n\nPrevious conversation:\n\n\nContext:\n\n'''Arya Stark''' is a fictional character in American author George R. R. Martin's ''A Song of Ice and Fire'' epic fantasy novel series.  She is a prominent point of view character in the novels with the third most viewpoint chapters, and is the only viewpoint character to have appeared in every published book of the series.\nIntroduced in 1996's ''A Game of Thrones'', Arya is the third child and younger daughter of Lord Eddard Stark and his wife Lady Catelyn Stark.  She is tomboyish, headstrong, feisty, independent, disdains traditional female pursuits, and is often mistaken for a boy.  She wields a smallsword named Needle, a gift from her half-brother, Jon Snow, and is trained in the Braavosi style of sword fighting by Syrio Forel.\nArya is portrayed by English actress Maisie Williams in HBO's Emmy-winning tel

In [58]:
list(set([i.meta["name"] for i in session.context_history]))




['30_List_of_A_Song_of_Ice_and_Fire_characters.txt',
 '43_Arya_Stark.txt',
 '371_Cersei_Lannister.txt']

In [ ]:
write a 3 paragraph summary of aria starks personality and drivers

how has her sister influenced her?

who influenced her more (both positive and negative) sansa or the hound?

a choice between the two